In [36]:
import importlib
import hashlib
import receiver # import gen_aes_key, elgamal_encrypt, aes_decrypt
import sender # import elgamal_keygen, elgamal_decrypt, aes_encrypt
importlib.reload(receiver)
importlib.reload(sender)

<module 'sender' from 'a:\\Ain Shams\\Academic\\Semester 9 Fall 2024\\CSE451 Computer and Network Security\\Project\\repo\\Secure-Messaging-App\\sender.py'>

# 1) Sender generate public+private Elgamal Key, and public+private RSA key


In [37]:
(sender_elgamal_public_p,sender_elgamal_public_g,sender_elgamal_public_y), sender_elgamal_private = sender.elgamal_keygen()
sender_RSA_public, sender_RSA_private = sender.generate_rsa_keys()

In [38]:
sender_elgamal_private

22445622597937175099087980002194291921512755574999096774160047761912329648835

In [39]:
sender_elgamal_public_p,sender_elgamal_public_g,sender_elgamal_public_y

(81159869763859104704902805864554706907994671739269514779254652557673574309373,
 20836739152337504219785357336163377408086265246853342662316004100451198910671,
 60229812477074072082687540485044723899221467695480121016080619628127603480579)

# 2) Sender publish public gamal key and public RSA key

In [40]:
# sender publish public elgamal and RSA keys

# 3) Receiver Generates AES Key

In [41]:
receiver_aes_key=receiver.gen_aes_key();
receiver_aes_key

b'\xe1\xed\x026R\xb3\xc1\x1c!hw\xab\x13L\x80\x9f'

# 4) Receiver Sends AES Key encrypted with elgamal

In [42]:
print(int.from_bytes(receiver_aes_key))
receiver_aes_encrypted = receiver.elgamal_encrypt((sender_elgamal_public_p,sender_elgamal_public_g,sender_elgamal_public_y),int.from_bytes(receiver_aes_key))
print(receiver_aes_encrypted)
#send to sender

300306918275813889087642041879553147039
(71203614604016133841187143428042878750772012639198408766185844983733133133877, 691570504958382825868406197960881678433276333564216331735791928537377159845)


# 5) Sender encrypt chat messages with AES + make SHA-256 hash of message

In [43]:
# sender decrypts aes key
receiver_aes_decrypted_at_sender = sender.elgamal_decrypt(sender_elgamal_private,(sender_elgamal_public_p,sender_elgamal_public_g,sender_elgamal_public_y),receiver_aes_encrypted)
print(receiver_aes_decrypted_at_sender)
receiver_aes_decrypted_at_sender_bytes = receiver_aes_decrypted_at_sender.to_bytes(length=16,byteorder='big') # receiver_aes_decrypted_at_sender.to_bytes((receiver_aes_decrypted_at_sender.bit_length() + 7) // 8, 'big')

# # Ensure the key is exactly 16 bytes long
# if len(receiver_aes_decrypted_at_sender_bytes) < 16:
#     # Pad with zero bytes to make it 16 bytes
#     receiver_aes_decrypted_at_sender_bytes = receiver_aes_decrypted_at_sender_bytes.rjust(16, b'\x00')
# elif len(receiver_aes_decrypted_at_sender_bytes) > 16:
#     # Truncate to the first 16 bytes
#     receiver_aes_decrypted_at_sender_bytes = receiver_aes_decrypted_at_sender_bytes[:16]
print(receiver_aes_decrypted_at_sender_bytes)
sender_message = "Message from sender"
sender_AES_encrypted_message = sender.aes_encrypt(receiver_aes_decrypted_at_sender_bytes,sender_message)
sender_message_bytes = sender_message.encode()
_SHA256_Hash = hashlib.sha256()
_SHA256_Hash.update(sender_message_bytes)
sender_message_bytes_SHA256_hashed = _SHA256_Hash.hexdigest()
print(sender_message_bytes_SHA256_hashed)

300306918275813889087642041879553147039
b'\xe1\xed\x026R\xb3\xc1\x1c!hw\xab\x13L\x80\x9f'
644237e3726e65b12cf5da5a74199df0e622740569600875b151c6cb231c7672


# 6) Sender signs hashed with private RSA + send to receiver

In [44]:
sender_message_bytes_SHA256_hashed_RSA_signed = sender.rsa_sign(sender_RSA_private,sender_message_bytes_SHA256_hashed)

# send sender_AES_encrypted_message, sender_message_bytes_SHA256_hashed_RSA_signed  to receiver

# 7) Receiver uses AES to decrypt message

In [45]:
receiver_AES_decrypted_msg = receiver.aes_decrypt(receiver_aes_key,sender_AES_encrypted_message)

# 8) Receiver uses RSA to verify that the message is sent by the sender

In [46]:
receiver_RSA_verified_hashed_msg = receiver.rsa_verify(sender_RSA_public,sender_message_bytes_SHA256_hashed_RSA_signed)



# 8) Receiver verifies hash and decrypts with AES

In [47]:
receiver_AES_decrypted_msg_bytes = receiver_AES_decrypted_msg.encode()
_SHA256_Hash = hashlib.sha256()
_SHA256_Hash.update(receiver_AES_decrypted_msg_bytes)
receiver_AES_decrypted_msg_bytes_SHA256_hashed = _SHA256_Hash.hexdigest()

# receiver_received_msg = receiver.aes_decrypt(receiver_aes_key,sender_encrypted_message)
print(receiver_AES_decrypted_msg_bytes_SHA256_hashed)
print(receiver_RSA_verified_hashed_msg)
print(receiver_RSA_verified_hashed_msg == receiver_AES_decrypted_msg_bytes_SHA256_hashed)

644237e3726e65b12cf5da5a74199df0e622740569600875b151c6cb231c7672
644237e3726e65b12cf5da5a74199df0e622740569600875b151c6cb231c7672
True


In [48]:
receiver_AES_decrypted_msg

'Message from sender'